In [ ]:
def kernel_init(shape):
    kernel = np.zeros(shape)
    kernel[:,:,0,0] = np.array([[1,0,-1],[1,0,-1],[1,0,-1]])
    return kernel

In [ ]:
model = Sequential()
model.add(Conv2D(1, [3,3], kernel_initializer=kernel_init, 
                 input_shape=(4,4,1), padding="valid"))
model.build()

In [ ]:
out = model.predict(image)
print(out[0,:,:,0])